# Using Translation Tool in a Chat
The [Translation tool](https://dev.writer.com/home/translation-tool) lets you translate text during a chat with a Palmyra LLM. While Palmyra X models can perform translation tasks, Palmyra Translate is a dedicated model optimized for translation use cases.

This cookbook shows how to use the Translation tool to perform translations in chat completions.

## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)


## Setup

Install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

The Translation tool follows this format:
```json
tools = [{
    "type": "translation",
    "function": {
        "model": "palmyra-translate",
        "formality": False,
        "length_control": False,
        "mask_profanity": True
    }
}]
```

The `model` field specifies which translation model to use (palmyra-translate).<br>
`formality` controls whether the translation should be formal (`true`) or neutral/informal (`false`).<br>
`length_control` can be used to influence the length of the translation output.<br>
`mask_profanity` ensures offensive words are masked in the translation.


In [ ]:
tools = [{
    "type": "translation",
    "function": {
        "model": "palmyra-translate",
        "formality": False,
        "length_control": False,
        "mask_profanity": True
    }
}]

The next code snippet asks you to input the text you want to translate, and then sends that text to the LLM. It passes the Translation tool to the LLM so the model can produce a translated response:

In [ ]:
text_to_translate = "Gestern ging ich im Park spazieren, sah viele bunte Blumen, hörte Vögel singen und traf einen alten Freund"

messages = [
    {"role": "user", "content": f"Please translate the following text:\n\n{text_to_translate}"}
]

response = client.chat.chat(
    messages=messages,
    model="palmyra-x5",
    tools=tools
)

print("Translated text:")
print(response.choices[0].message.content)


## Streaming

You can also stream the chat response, which will stream the response as it is generated. The following code snippet shows how to stream the response, collect the text as it is generated, and print it out.

In [ ]:
response_stream = client.chat.chat(
    model="palmyra-x5",
    messages=[{"role": "user", "content": text_to_translate}],
    tools=tools,
    tool_choice="auto",
    stream=True
)

output_text = ""

for chunk in response_stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        output_text += chunk.choices[0].delta.content

print(f"\nFinal output: {output_text}")